In [ ]:
# Importando as bibliotecas
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import scipy.stats as stats
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import RANSACRegressor
from sklearn.ensemble import ExtraTreesRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.svm import SVR
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from scipy.stats import randint
from sklearn.ensemble import IsolationForest, ExtraTreesRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

In [ ]:
# Carregando o Dataset
url = 'https://raw.githubusercontent.com/klaytoncastro/idp-machinelearning/refs/heads/main/airquality/AirQualityUCI.csv'
df = pd.read_csv(url, delimiter = ';', decimal = ',')
df.head()

In [ ]:
# Verificando a estrutura de dados
df.info()

In [ ]:
# Convertendo as colunas Date e Time para DateTime
df['DateTime'] = pd.to_datetime(df['Date'] + ' ' + df['Time'], format='%d/%m/%Y %H.%M.%S')

In [ ]:
# Removendo as colunas originais Date e Time
df.drop(columns=['Date', 'Time'], inplace=True)
df.head()

In [ ]:
df.info()

In [ ]:
df.head()

In [ ]:
# Calculando as correlações entre as variáveis ​​preditoras e a variável alvo
correlations = df.corr()['CO(GT)'].sort_values(ascending=False)
print(correlations)

In [ ]:
# Verificando a distribuição dos dados
df.describe()

In [ ]:
# Identificamos um padrão estranho, onde -200 aparece como valor mínimo para cada uma das variáveis.
# Por isso, vamos contar valores -200 em cada coluna e avaliar se isso é frequente ou eventual.
print("Contagem de valores -200 em cada coluna:")
for column in df.columns:
    count_negative_200 = (df[column] == -200).sum()
    print(f"{column}: {count_negative_200}")

In [ ]:
# De fato, são valores anômalos. Vamos substituir -200 por NaN (NULL)
df.replace(-200, np.nan, inplace=True)
df.describe()

In [ ]:
# Verificando quantidade de missing values por coluna
missing_values = df.isna().sum().div(df.shape[0]).to_frame().sort_values(by=0, ascending=False)
missing_values.plot(kind='bar', figsize=(10, 5))
plt.title('Porcentagem de valores ausentes por coluna')
plt.show()

In [ ]:
# Decidimos a descartar a coluna NMHC(GT), mais 80% de valores ausentes. Imputar a mediana pode apresentar padrões lineares artificiais.
df.drop(columns=['NMHC(GT)'], inplace=True)
df.describe()

In [ ]:
# Criando uma nova coluna para agregar os valores e gerando um gráfico de barras dos total de valores ausentes por registro único
df['missing_values'] = df.isnull().any(axis=1)
df.groupby('missing_values').size().plot(kind='bar')
plt.title('Número de valores ausentes por observação')
plt.show()

In [ ]:
df.head()

In [ ]:
# Hipótese 1: se descartarmos as demais colunas ou registros com valores nulos, perderemos muita capacidade de previsão do modelo.
# Poderiamos preencher os valores NaN com a mediana da coluna, mas os padrões lineares gerados seriam de fato artificiais.
# Dessa forma, vamos seguir com a Hipótese 2.
for column in df.columns:
    if df[column].isnull().any():
        df[column].fillna(df[column].median(), inplace=True)

In [ ]:
df.describe()

In [ ]:
# Hipótese 2: Como ainda teremos em torno de 7000 observações na amostra após remover os dados ausentes, decidimos removê-los para assegurar maior fidelidade.
df = df.dropna()
df.describe()

In [ ]:
# As distribuições parecem melhores agora. Vamos exibir a nova matriz de correlação para análise.

In [ ]:
correlation_matrix = df.corr()

In [ ]:
plt.figure(figsize=(12, 8))
sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm', fmt='.2f', linewidths=0.5, vmin = -1)
plt.title('Correlation Matrix Heatmap', fontsize=16)
plt.show()

In [ ]:
# Vamos excluir a coluna intermediária 'missing_values' e a coluna 'DateTime'
df.drop(columns=['DateTime'], inplace=True)
#df.drop(columns=['missing_values'], inplace=True)

In [ ]:
# Vamos manter as demais variáveis de baixa correlação por enquanto.
df.drop(columns=['T'], inplace=True)
df.drop(columns=['RH'], inplace=True)
df.drop(columns=['AH'], inplace=True)

In [ ]:
df.head()

In [ ]:
# Calculando as correlações entre as variáveis ​​preditoras e a variável alvo
correlations = df.corr()['CO(GT)'].sort_values(ascending=False)
print(correlations)

In [ ]:
# Preparando as variáveis para treinar o modelo.
X = df.drop('CO(GT)', axis=1)
y = df['CO(GT)']

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
# Treinar com o ExtraTrees
#model = ExtraTreesRegressor(random_state=7, n_estimators=67, max_features='sqrt', max_depth=100, min_samples_split=13, min_samples_leaf=1, bootstrap = False)
#model = ExtraTreesRegressor(random_state=42, n_estimators=350, max_features='sqrt', max_depth=None, min_samples_split=2, min_samples_leaf=1)
model = ExtraTreesRegressor();
model.fit(X_train, y_train)

In [ ]:
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

In [ ]:
# Fazer previsões
y_pred = model.predict(X_test)

In [ ]:
# Calcular métricas
mae = mean_absolute_error(y_test, y_pred)
mse = mean_squared_error(y_test, y_pred)
rmse = np.sqrt(mse)
r2 = r2_score(y_test, y_pred)

In [ ]:
print(f'Mean Absolute Error (MAE): {mae}')
print(f'Mean Squared Error (MSE): {mse}')
print(f'Root Mean Squared Error (RMSE): {rmse}')
print(f'R² Score: {r2}')

In [ ]:
df.describe()

In [ ]:
# Gráfico de valores previstos x valores atuais
plt.figure(figsize=(10, 6))
plt.scatter(x=y_test, y=y_pred, color='blue', label='Valores Previstos')
plt.plot([y_test.min(), y_test.max()], [y_test.min(), y_test.max()], color='red', linestyle='-', label='Ideal Line')
plt.xlabel('Valores reais')
plt.ylabel('Valores Previstos')
plt.title('Valores previstos x Valores reais')
plt.legend()
plt.show()